In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,  KFold, cross_val_score
from sklearn.neural_network import MLPRegressor

In [3]:
train = pd.read_csv('../data/data3_train.csv')
test = pd.read_csv('../data/data3_test.csv')

In [4]:
target_column = "shot_made_flag"  # y_column_name

In [5]:
def split_x_y(df, target):
    y = df.loc[:, [target]]
    x = df.drop([target], axis=1)
    return x, y

def split_regular_playoff(df, season, train=None, test=None, validation=None):

    # if(train + test + validation != 1):
    #     print("Error dataset split")
    #     return None

    df_validaiton = None

    df_season = df[df['season'] == season]
    isPlayoffs = df_season.playoffs == 1

    df_train = df_season[~isPlayoffs]
    df_test  = df_season[isPlayoffs]

    print(f"Size of training dataset {len(df_train)}" )
    print(f"Size of test dataset {len(df_test)}" )
    print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )

    return df_train, df_test
#     train = split_x_y(df_train, 'shot_made_flag')
#     test = split_x_y(df_test, 'shot_made_flag')
#     return train[0], train[1], test[0], test[1]
#     return train, test

# x_train, y_train, x_test, y_test = split_regular_playoff(df, '2010-11')
# train, test = split_regular_playoff(df, '2010-11')

In [6]:
train.head()

,loc_x,loc_y,period,playoffs,shot_distance,shot_made_flag,shot_zone_basic,shot_zone_range,time_remaining,last_5_games_avg,...,shot_type_2PT Field Goal,shot_type_3PT Field Goal,shot_zone_area_Back Court(BC),shot_zone_area_Center(C),shot_zone_area_Left Side Center(LC),shot_zone_area_Left Side(L),shot_zone_area_Right Side Center(RC),shot_zone_area_Right Side(R),matchup_away,matchup_home
0,0,0,3,0,0,0.0,0,0,225,0.0,...,1,0,0,1,0,0,0,0,0,1
1,0,0,3,0,0,0.0,0,0,429,0.0,...,1,0,0,1,0,0,0,0,0,1
2,144,41,3,0,14,1.0,2,1,159,0.0,...,1,0,0,0,0,0,0,1,0,1
3,-161,132,3,0,20,0.0,2,2,49,0.0,...,1,0,0,0,1,0,0,0,0,1
4,34,82,4,0,8,0.0,1,1,596,0.0,...,1,0,0,1,0,0,0,0,0,1


## Split X & Y

In [48]:
x_train, y_train = split_x_y(train, target_column)
x_test, y_test = split_x_y(test, target_column)

In [9]:
lambdas = [1e-10,1e-5,1e-4,1e-3,1e-2,0.1, 0.5,1,5,10,50,100]

# Linear Regression

In [19]:
reg = LinearRegression()
reg.fit(x_train, y_train)
reg.score(x_train, y_train)

0.2299316121604148

#### train data

In [52]:
y_train_pred = reg.predict(x_train)
y_train_pred = y_train_pred >= 0.6
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

0.7150297619047619

#### Test Data

In [58]:
y_test_pred = reg.predict(x_test)
y_test_pred = y_test_pred >= 0.5
y_test_pred = np.multiply(y_test_pred, 1)
accuracy_score(y_test.values, y_test_pred)

0.6799065420560748

# Lasso Regression

In [15]:
y_train.values.flatten()

array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])

In [19]:
lasso_cv = LassoCV(alphas=lambdas, cv=5, max_iter=100_000)
lasso_cv.fit(x_train.values, y_train.values.flatten())

lasso_r2 =  np.mean(cross_val_score(lasso_cv, x_train.values, y_train.values.flatten()))

print('Best lambda:', lasso_cv.alpha_, 'R2 score:',lasso_r2)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.23965911904753057, tolerance: 0.026618604651162787
  model = cd_fast.enet_coordinate_descent_gram(
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 33.88716132641106, tolerance: 0.026703345724907065
  model = cd_fast.enet_coordinate_descent_gram(
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.7471318485644645, tolerance: 0.02138093023255813
  model = cd_fast.enet_coordinate_descent_gram(
/home/teemo/anaconda3/envs/ds/lib/pyth

Best lambda: 0.001 R2 score: 0.1925524666578287


/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.167473087145524, tolerance: 0.02131521486643436
  model = cd_fast.enet_coordinate_descent_gram(


In [30]:
y_train_pred = lasso_cv.predict(x_train)
y_train_pred = y_train_pred >= 0.5
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LassoCV was fitted without feature names
  warnings.warn(


0.7120535714285714

In [51]:
y_test_pred = lasso_cv.predict(x_test)
y_test_pred = y_test_pred >= 0.5
y_test_pred = np.multiply(y_test_pred, 1)
accuracy_score(y_test.values, y_test_pred)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LassoCV was fitted without feature names
  warnings.warn(


0.6869158878504673

# Ridge Regression

In [32]:
ridge_cross_val_metrics = pd.DataFrame(columns=['mean MSE', 'mean norm_MSE', 'mean R2'])
# We calculate the cross-validation metrics for each lambda 
for lambda_val in lambdas:
    kf = KFold(n_splits=5)
    i=1
    cv_mse = []
    cv_nmse = []
    cv_r2 = []
    # We compute the metrics for each fold and then perform the mean.
    for train_index, test_index in kf.split(x_train):
        x_train_fold = x_train.values[train_index]
        y_train_fold = y_train.values[train_index]
        x_test_fold = x_train.values[test_index,:]
        y_test_fold = y_train.values[test_index]

        ridge = Ridge(alpha=lambda_val)
        ridge.fit(x_train_fold,y_train_fold)
        y_pred_fold = ridge.predict(x_test_fold)
        fold_mse = mean_squared_error(y_test_fold, y_pred_fold)
        fold_nmse =  1-r2_score(y_test_fold, y_pred_fold)
        fold_r2 = r2_score(y_test_fold, y_pred_fold)
        cv_mse.append(fold_mse)
        cv_nmse.append(fold_nmse)
        cv_r2.append(fold_r2)
    ridge_cross_val_metrics.loc['Lambda={}'.format(lambda_val),:] = [np.mean(cv_mse),np.mean(cv_nmse),np.mean(cv_r2)]

ridge_cross_val_metrics.loc['Mean', :] = ridge_cross_val_metrics.mean()
ridge_cv_r2 = ridge_cross_val_metrics.loc["Mean", "mean R2"]

ridge_cross_val_metrics.sort_values(by='mean R2',ascending=False)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.00747e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.07506e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.01917e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.03564e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/teemo/anaconda3/en

,mean MSE,mean norm_MSE,mean R2
Lambda=5,0.20011,0.810713,0.189287
Lambda=10,0.200881,0.813862,0.186138
Lambda=1,0.201261,0.815277,0.184723
Lambda=0.5,0.201932,0.817959,0.182041
Lambda=0.1,0.202772,0.821318,0.178682
Lambda=0.01,0.203019,0.822308,0.177692
Lambda=0.001,0.203045,0.822413,0.177587
Lambda=0.0001,0.203048,0.822424,0.177576
Lambda=1e-05,0.203048,0.822425,0.177575
Lambda=1e-10,0.203049,0.822426,0.177574


In [43]:
temp_df = ridge_cross_val_metrics.sort_values(by='mean R2',ascending=False)
temp_df.iloc[0]

mean MSE          0.20011
mean norm_MSE    0.810713
mean R2          0.189287
Name: Lambda=5, dtype: object

In [53]:
ridge = Ridge(alpha=5)

ridge.fit(x_train.values, y_train.values.flatten())

y_train_pred = ridge.predict(x_train)
y_train_pred = y_train_pred >= 0.5
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


0.7142857142857143

In [54]:
y_test_pred = ridge.predict(x_test)
y_test_pred = y_test_pred >= 0.5
y_test_pred = np.multiply(y_test_pred, 1)
accuracy_score(y_test.values, y_test_pred)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


0.6845794392523364

# Neural Network

In [71]:
y_train.values.flatten()

array([0., 0., 1., ..., 0., 1., 1.])

In [145]:
regr = MLPRegressor(hidden_layer_sizes=(35, 15, 5), max_iter=800_000, tol=1e-8)
regr.fit(x_train.values, y_train.values.flatten())

MLPRegressor(hidden_layer_sizes=(35, 15, 5), max_iter=800000, tol=1e-08)

In [146]:
y_train_pred = regr.predict(x_train.values)
nn_score = regr.score(x_train.values, y_train)
nn_mse = mean_squared_error(y_train, y_pred)
nn_r2 = r2_score(y_train, y_pred)

In [147]:
y_train_pred = regr.predict(x_train.values)
y_train_pred = y_train_pred >= 0.5
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

0.5699404761904762

In [148]:
y_test_pred = regr.predict(x_test.values)
y_test_pred = y_test_pred >= 0.5
y_test_pred = np.multiply(y_test_pred, 1)
accuracy_score(y_test.values, y_test_pred)

0.5514018691588785

In [23]:
target_encoding(train, "action_type", target_column)

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,matchup,opponent,shot_id,time_remaining,last_5_games_avg,streak_before_shot,points_before_shot,fgp_before_shot,month,action_type_te
0,Driving Layup Shot,Layup,584,21000003,34.0213,18,23,-118.2518,4,0,...,home,HOU,16453,105,0.39,0,10,0.36,10,0.706897
1,Jump Shot,Jump Shot,538,21000003,33.9173,-79,127,-118.3488,4,0,...,home,HOU,16452,259,0.39,0,10,0.38,10,0.326633
2,Jump Shot,Jump Shot,350,21000003,33.9363,-180,108,-118.4498,3,0,...,home,HOU,16450,400,0.39,2,10,0.42,10,0.326633
3,Jump Shot,Jump Shot,332,21000003,33.8983,-127,146,-118.3968,3,0,...,home,HOU,16449,493,0.39,1,8,0.36,10,0.326633
4,Jump Shot,Jump Shot,322,21000003,33.9623,-106,82,-118.3758,3,0,...,home,HOU,16448,546,0.39,0,6,0.30,10,0.326633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,Jump Shot,Jump Shot,468,21001230,33.9943,-98,50,-118.3678,4,0,...,away,SAC,18067,89,0.42,0,22,0.69,4,0.326633
1356,Jump Shot,Jump Shot,475,21001230,33.9553,144,89,-118.1258,4,0,...,away,SAC,18068,57,0.42,0,22,0.65,4,0.326633
1357,Jump Shot,Jump Shot,480,21001230,33.9963,-119,48,-118.3888,4,0,...,away,SAC,18069,15,0.42,0,22,0.61,4,0.326633
1358,Jump Shot,Jump Shot,506,21001230,33.9153,92,129,-118.1778,5,0,...,away,SAC,18071,244,0.42,1,25,0.60,4,0.326633


## Fit Linear Regression